In [1]:
import json
import geopandas as gpd
from shapely.geometry import MultiPoint
import random
import folium
import folium.plugins as plugins
from shapely.geometry import MultiPoint
import pandas as pd
from shapely.geometry import Point
from folium import TileLayer

In [2]:
# Langkah 1: Baca file CSV menggunakan pandas
df = pd.read_csv('../data/koordinat_masjid.csv')

# Langkah 2: Konversi DataFrame ke GeoDataFrame
# Misalkan kolom latitude dan longitude di CSV bernama 'lat' dan 'long'
geometry = [Point(xy) for xy in zip(df['long'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf.crs = 'EPSG:4326'
# gdf['centroid_latitude'] = gdf['geometry'].centroid.y
# gdf['centroid_longitude'] = gdf['geometry'].centroid.x
# gdf['geometry'].centroid.to_list()
gdf

,nama,lat,long,geometry
0,Al Ikhlas Bone,0.33756,123.46097,POINT (123.46097 0.33756)
1,Al Ihsan Bone Raya,0.31044,123.34598,POINT (123.34598 0.31044)
2,At Taibah Bulawa,0.32442,123.27561,POINT (123.27561 0.32442)
3,Al Arqam Suwawa Timur,0.51472,123.22386,POINT (123.22386 0.51472)
4,Nurul Ihklas Bone Pantai,0.38017,123.21278,POINT (123.21278 0.38017)
...,...,...,...,...
78,Salafiyah Syafi’iyah Taluditi,0.54806,121.79000,POINT (121.79 0.54806)
79,Arrayan Wanggarasi,0.53619,121.70239,POINT (121.70239 0.53619)
80,Al Musyawwarah Lemito,0.54458,121.57389,POINT (121.57389 0.54458)
81,An-Nuur Popayato Timur,0.51033,121.47467,POINT (121.47467 0.51033)


In [3]:
df

,nama,lat,long
0,Al Ikhlas Bone,0.33756,123.46097
1,Al Ihsan Bone Raya,0.31044,123.34598
2,At Taibah Bulawa,0.32442,123.27561
3,Al Arqam Suwawa Timur,0.51472,123.22386
4,Nurul Ihklas Bone Pantai,0.38017,123.21278
...,...,...,...
78,Salafiyah Syafi’iyah Taluditi,0.54806,121.79000
79,Arrayan Wanggarasi,0.53619,121.70239
80,Al Musyawwarah Lemito,0.54458,121.57389
81,An-Nuur Popayato Timur,0.51033,121.47467


In [7]:
import plotly.graph_objects as go
import pandas as pd
import random

# Titik koordinat Ka'bah
kabah_coords = [39.8262, 21.4225]  # [longitude, latitude]

# Contoh data masjid (ganti dengan data asli Anda)
# data = {
#     'nama': ['Masjid A', 'Masjid B', 'Masjid C'],
#     'lat': [21.4225, 21.4226, 21.4227],
#     'long': [39.8262, 39.8263, 39.8264]
# }
# gdf = pd.DataFrame(data)

# Function to generate a random color
def random_color():
    return '#{:06x}'.format(random.randint(0, 0xFFFFFF))

# Create a dictionary mapping each nama to a random color
unique_distrik = gdf['nama'].unique()
color_dict = {distrik: random_color() for distrik in unique_distrik}

# Create traces for each mosque
traces = []
lines = []

for _, row in gdf.iterrows():
    # Add scatter trace for each mosque
    traces.append(go.Scattergeo(
        lon=[row['long']],
        lat=[row['lat']],
        mode='markers+text',
        marker=dict(
            size=10,
            color=color_dict[row['nama']],
            symbol='circle'
        ),
        text=row['nama'],
        textposition='top center',
        name=row['nama']
    ))
    
    # Add line trace from each mosque to Ka'bah
    lines.append(go.Scattergeo(
        lon=[row['long'], kabah_coords[0]],
        lat=[row['lat'], kabah_coords[1]],
        mode='lines',
        line=dict(
            width=2,
            color='red',
            dash='dash'
        ),
        showlegend=False
    ))

# Create the map
fig = go.Figure()

# Add the traces to the figure
for trace in traces:
    fig.add_trace(trace)

for line in lines:
    fig.add_trace(line)

# Update the layout
fig.update_layout(
    title='Peta Globe 3D dengan Titik Masjid dan Garis ke Ka\'bah',
    geo=dict(
        scope='world',
        projection_type='orthographic',
        showland=True,
        landcolor='rgb(243, 243, 243)',
        showocean=True,
        oceancolor='rgb(204, 204, 255)',
        showlakes=True,
        lakecolor='rgb(255, 255, 255)',
        showcountries=True,
        countrycolor='rgb(204, 204, 204)'
    ),
    showlegend=True
)

# fig.show()
# Save the figure to an HTML file
fig.write_html('../web/map-html/globe.html', auto_open=True)
